In [1]:
import tensorflow.keras
from unet import unet_model_3d


In [2]:
input_shape = (1,512,512,95)
input_shape = (1,40,40,40)
model = unet_model_3d(input_shape = input_shape, n_labels = 1, activation_name = 'relu')
#model.summary()

In [3]:
from data import create_dataset, patches_dataset

batch_size = 5
path_images = '/home/jaume/Documentos/DL_postgraduate/final_project/sample_data/resized_training/'
path_labels = '/home/jaume/Documentos/DL_postgraduate/final_project/sample_data/resized_targets/'

train_dataset = patches_dataset(batch_size,path_images,path_labels,(40,40,40))

In [4]:
for image in train_dataset:
    x,y = image
    print(x.shape)
    print(y.shape)
    break

(5, 1, 40, 40, 40)
(5, 1, 40, 40, 40)


In [5]:
model.fit(train_dataset,epochs=2,verbose=1)

Epoch 1/2
      1/Unknown - 39s 39s/step - loss: -4.3620e-08 - dice_coefficient: 4.3620e-08